## IAC Depths 

This jupiter notebook splits a line feature into points at 5-meter intervals, adds an attribute value, and then extracts depths at those point locations from a bathymetry raster:

In [1]:
from arcgis.features import FeatureLayer, FeatureSet
from arcgis.geometry import Point
import arcpy

#### 1. Load the shapefile and raster data

Replace "path/to/your/feature_class.shp" with the path to your specific feature class. Adjust as needed depending on the format and location of your feature class.

In [ ]:

# Define paths
input_line_feature = r"path/to/your/line_feature.shp"  # Replace with your line feature path
bathymetry_raster = r"path/to/your/bathymetry_raster.tif"  # Replace with your raster path


In [ ]:
# Define the attribute value you want to add to the points
attribute_value = "Depth"  # Replace with the desired attribute value


### 2.  Split line to points with depths information added 
- 1) Adjust the attribute value as needed.
- 2) Splot plit the line into points at 5-meter intervals,
- 3) Assign the specified attribute value, and extract depth values from the raster at those points' locations.
- 4) Adjust as necessary based on your specific data and requirements.

In [ ]:
def split_line_to_points_with_depths(input_feature, raster, attribute):
    # Create a cursor to iterate through the line features
    with arcpy.da.SearchCursor(input_feature, ["SHAPE@", "OID@"]) as cursor:
        for row in cursor:
            # Get the geometry (line) and ObjectID from the feature
            line_geometry = row[0]
            feature_oid = row[1]
            
            # Initialize variables for the loop
            distance_along_line = 0
            increment = 5  # 5-meter intervals
            
            # Create a while loop to split the line into points at 5-meter intervals
            while distance_along_line < line_geometry.length:
                # Create a point at the given distance along the line
                point = line_geometry.positionAlongLine(distance_along_line)
                
                # Create a new point feature
                output_point = arcpy.PointGeometry(point, line_geometry.spatialReference)
                
                # Add the attribute value to the point feature
                attributes = [(feature_oid, attribute)]
                arcpy.management.AddFields(output_point, [["Feature_OID", "LONG"], [attribute, "DOUBLE"]])
                with arcpy.da.InsertCursor(output_point, ["Feature_OID", attribute]) as icursor:
                    icursor.insertRow(attributes)
                
                # Extract depth value from the raster at the point location
                depth_result = arcpy.GetCellValue_management(raster, point)
                depth_value = float(depth_result.getOutput(0))
                
                # Update the attribute value for the point
                with arcpy.da.UpdateCursor(output_point, [attribute]) as ucursor:
                    for row in ucursor:
                        row[0] = depth_value
                        ucursor.updateRow(row)
                
                # Save the point as a feature class with attribute value
                output_name = f"Point_{feature_oid}_5m"  # Naming convention for output points
                arcpy.CopyFeatures_management(output_point, output_name)
                
                # Move along the line by 5 meters
                distance_along_line += increment
    
    print("Splitting and attribute assignment completed.")

# Call the function
split_line_to_points_with_depths(input_line_feature, bathymetry_raster, attribute_value)